In [1]:
# !pip install --upgrade mxnet
# !pip install autogluon

In [2]:
from autogluon.text import TextPredictor
import pandas as pd

train_data = pd.read_csv('sub_train.csv').drop(columns=['train_id','name','item_condition_id','shipping','category_name','item_description'])
test_data = pd.read_csv('sub_val.csv')
test_data_nolab = test_data.drop(columns=['price','test_id']) 
time_limit = 1 * 60  # set to larger value in your applications

In [4]:
train_data

,brand_name,price
0,Sonoma,24.0
1,Tobi,9.0
2,NaN,16.0
3,Old Navy,8.0
4,Schick,9.0
...,...,...
14820,NaN,8.0
14821,NaN,8.0
14822,NaN,20.0
14823,NaN,35.0


In [3]:
length=0
for item in train_data['brand_name'].apply(str):
    length+=len(item)/len(train_data['brand_name'])
print(length)

6.368836424957822


In [5]:
import os
import time
import os
import psutil
def count_time(func):
    def int_time():
        start_time = time.time()
        func()
        over_time = time.time()
        total_time = over_time - start_time
        print("程序运行了%s秒" % total_time)
    return int_time

def count_info(func):
    def float_info():
        pid = os.getpid()
        p = psutil.Process(pid)
        info_start = p.memory_full_info().uss/1024
        func()
        info_end=p.memory_full_info().uss/1024
        print("程序占用了内存"+str(info_end-info_start)+"KB")
    return float_info

In [6]:
@count_time
@count_info
def calcu_atg():
    predictor = TextPredictor(label='price', path='autogluon')
    predictor.fit(train_data, time_limit=time_limit)
    embeddings = predictor.extract_embedding(train_data)
    embeddings.shape
    
calcu_atg()

Global seed set to 123
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type                         | Params
-------------------------------------------------------------------
0 | model             | HFAutoModelForTextPrediction | 108 M 
1 | validation_metric | MeanSquaredError             | 0     
2 | loss_func         | MSELoss                      | 0     
-------------------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
217.785   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 123


Training: 0it [00:00, ?it/s]

/root/miniconda3/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Epoch 0, global step 52: val_rmse reached 0.74201 (best 0.74201), saving model to "/root/autodl-tmp/mercari-price-suggestion-challenge/autogluon/epoch=0-step=52.ckpt" as top 3
Time limit reached. Elapsed time is 0:01:00. Signaling Trainer to stop.


Validating: 0it [00:00, ?it/s]

Epoch 0, global step 79: val_rmse reached 0.74709 (best 0.74201), saving model to "/root/autodl-tmp/mercari-price-suggestion-challenge/autogluon/epoch=0-step=79.ckpt" as top 3


Predicting: 0it [00:00, ?it/s]

程序占用了内存6931976.0KB
程序运行了105.09703803062439秒
